In [69]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
import yaml
import os
import pandas as pd
import numpy as np
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier


In [35]:
datadir = os.path.join(os.getcwd(),'data/donors-choose-splits/')
traindf = pd.read_csv(os.path.join(datadir,'train_20120501_20120801.csv.gz'), compression='gzip')
testdf = pd.read_csv(os.path.join(datadir,'test_20121201_20130201.csv.gz'), compression='gzip')
train_attrdf = pd.read_csv(os.path.join(datadir,'train_20120501_20120801_protected.csv.gz'), compression='gzip')
test_attrdf = pd.read_csv(os.path.join(datadir,'test_20121201_20130201_protected.csv.gz'), compression='gzip')


In [66]:
traindf.shape

(16790, 113)

In [36]:
train_attrdf['poverty_level'].value_counts()

highest    9448
lower      7342
Name: poverty_level, dtype: int64

In [54]:
traindf[train_attrdf['poverty_level']=='highest']['quickstart_label'].sum() / 9448

0.338272650296359

In [53]:
traindf[train_attrdf['poverty_level']=='lower']['quickstart_label'].sum() / 7342

0.42631435576137294

In [61]:
train_poverty_highest = traindf[train_attrdf['poverty_level']=='highest'].copy()

sample_train_poverty_highest = train_poverty_highest.sample(n=7342, replace=False, random_state=1)

In [62]:
sample_train_poverty_highest.shape

(7342, 113)

In [63]:
sample_train_poverty_highest['quickstart_label'].sum() / len(sample_train_poverty_highest)

0.336692999182784

In [ ]:
new_traindf = 

In [64]:
new_traindf = pd.concat([sample_train_poverty_highest, traindf[train_attrdf['poverty_level']=='lower'].copy()], axis=0)


In [65]:
new_traindf.shape

(14684, 113)

In [70]:
datadir = os.path.join(os.getcwd(),'data/donors-choose/model_selection')
evals_df = pd.read_csv(os.path.join(datadir,'split2_evals.csv'))

In [71]:
evals_df[evals_df['model_uuid']=='a04e2eedd9c5ff18bcf77e84ae9db561']

,model_precision,model_classpath,hyperparameters,model_uuid,predictions_uuid,target_pp,matrix_type,matrix_start_date,matrix_end_date
0,0.552,sklearn.ensemble.RandomForestClassifier,"{""n_jobs"": -1, ""criterion"": ""gini"", ""max_depth...",a04e2eedd9c5ff18bcf77e84ae9db561,c598fbe93f4c218ac7d325fb478598f1,1000,test,2012-12-01,2013-01-31


In [72]:
import ast
hyperparameters= ast.literal_eval(evals_df['hyperparameters'][0])

In [73]:
rf = RandomForestClassifier(**hyperparameters)

In [74]:
rf.__dict__

{'base_estimator': DecisionTreeClassifier(),
 'n_estimators': 87,
 'estimator_params': ('criterion',
  'max_depth',
  'min_samples_split',
  'min_samples_leaf',
  'min_weight_fraction_leaf',
  'max_features',
  'max_leaf_nodes',
  'min_impurity_decrease',
  'min_impurity_split',
  'random_state',
  'ccp_alpha'),
 'bootstrap': True,
 'oob_score': False,
 'n_jobs': -1,
 'random_state': 213500298,
 'verbose': 0,
 'warm_start': False,
 'class_weight': None,
 'max_samples': None,
 'criterion': 'gini',
 'max_depth': 30,
 'min_samples_split': 3,
 'min_samples_leaf': 44,
 'min_weight_fraction_leaf': 0.0,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'ccp_alpha': 0.0}

In [75]:
y_train = new_traindf['quickstart_label'].values

In [76]:
rf.fit(new_traindf.drop(['entity_id','as_of_date','quickstart_label'], axis = 1), y_train)

RandomForestClassifier(max_depth=30, max_features='sqrt', min_samples_leaf=44,
                       min_samples_split=3, n_estimators=87, n_jobs=-1,
                       random_state=213500298)

In [77]:
y_pred = rf.predict_proba(testdf.drop(['entity_id','as_of_date','quickstart_label'], axis = 1))[:,1]

In [79]:
new_preds = testdf[['entity_id','as_of_date','quickstart_label']].copy()

In [80]:
new_preds['predict_proba'] = y_pred

In [81]:
new_preds.head()

,entity_id,as_of_date,quickstart_label,predict_proba
0,185730,1354320000000,0.0,0.324587
1,185729,1354320000000,0.0,0.346998
2,185738,1354320000000,0.0,0.450499
3,185737,1354320000000,0.0,0.472256
4,185736,1354320000000,0.0,0.254969


In [83]:
split1_attrdf

In [108]:
new_preds[new_preds['score'] > 0]['quickstart_label'].sum() / 1000

0.522

In [94]:
df = pd.merge(new_preds, split1_attrdf, how='left', on=['entity_id','as_of_date'], left_index=True, right_index=False, sort=True, copy=True)

In [97]:
df = df.rename(columns = {'quickstart_label':'label_value'})

In [98]:
df.head()

,entity_id,as_of_date,label_value,predict_proba,score,poverty_level,teacher_sex,metro_type
17310,119,1359504000000,0.0,0.360968,0.0,lower,female,suburban_rural
15801,120,1359158400000,0.0,0.320894,0.0,highest,female,unknown
14694,121,1358812800000,1.0,0.492224,0.0,lower,female,suburban_rural
14226,122,1358726400000,0.0,0.418132,0.0,lower,female,unknown
14227,123,1358726400000,0.0,0.315036,0.0,lower,female,urban


In [99]:
g = Group()
xtab, _ = g.get_crosstabs(df[['score','label_value','poverty_level','metro_type', 'teacher_sex']].copy())

model_id, score_thresholds 0 {'rank_abs': [1000]}


In [100]:
absolute_metrics = g.list_absolute_metrics(xtab)

In [101]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

,model_id,score_threshold,k,attribute_name,attribute_value,pp,pn,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities
0,0,binary 0/1,1000,poverty_level,highest,189,9784,73,2996,6788,116,3112,6861,9973,17677
1,0,binary 0/1,1000,poverty_level,lower,811,6893,405,2452,4441,406,2858,4846,7704,17677
2,0,binary 0/1,1000,metro_type,suburban_rural,671,5835,335,2057,3778,336,2393,4113,6506,17677
3,0,binary 0/1,1000,metro_type,unknown,174,1882,83,674,1208,91,765,1291,2056,17677
4,0,binary 0/1,1000,metro_type,urban,155,8960,60,2717,6243,95,2812,6303,9115,17677
5,0,binary 0/1,1000,teacher_sex,female,919,14174,434,4662,9512,485,5147,9946,15093,17677
6,0,binary 0/1,1000,teacher_sex,male,81,2503,44,786,1717,37,823,1761,2584,17677


In [102]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics]

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,poverty_level,highest,0.037275,0.989360,0.306214,0.386243,0.010640,0.962725,0.693786,0.613757,0.189,0.018951,0.312043
1,poverty_level,lower,0.142057,0.916426,0.355723,0.499383,0.083574,0.857943,0.644277,0.500617,0.811,0.105270,0.370976
2,metro_type,suburban_rural,0.140410,0.918551,0.352528,0.499255,0.081449,0.859590,0.647472,0.500745,0.671,0.103136,0.367814
3,metro_type,unknown,0.118954,0.935709,0.358130,0.477011,0.064291,0.881046,0.641870,0.522989,0.174,0.084630,0.372082
4,metro_type,urban,0.033784,0.990481,0.303237,0.387097,0.009519,0.966216,0.696763,0.612903,0.155,0.017005,0.308502
5,teacher_sex,female,0.094230,0.956364,0.328912,0.472252,0.043636,0.905770,0.671088,0.527748,0.919,0.060889,0.341019
6,teacher_sex,male,0.044957,0.975014,0.314023,0.543210,0.024986,0.955043,0.685977,0.456790,0.081,0.031347,0.318498


In [178]:
label_pos_poverty_highest = traindf.loc[(train_attrdf['poverty_level']=='highest') & traindf['quickstart_label'] > 0]
label_neg_poverty_highest = traindf.loc[(train_attrdf['poverty_level']=='highest') & traindf['quickstart_label'] == 0]

In [179]:
label_pos_poverty_lower = traindf.loc[(train_attrdf['poverty_level']=='lower') & traindf['quickstart_label'] > 0]
label_neg_poverty_lower = traindf.loc[(train_attrdf['poverty_level']=='lower') & traindf['quickstart_label'] == 0]

In [111]:
label_pos_poverty_highest.shape

(3196, 113)

In [112]:
label_pos_poverty_lower.shape

(3130, 113)

In [122]:
label_neg_poverty_highest.shape

(13594, 113)

In [123]:
label_neg_poverty_lower.shape

(13660, 113)

In [266]:
sample_pos_poverty_highest = label_pos_poverty_highest.sample(n=3000, replace=False, random_state=1)
sample_neg_poverty_highest = label_neg_poverty_highest.sample(n=3000, replace=False, random_state=1)

sample_pos_poverty_lower = label_pos_poverty_lower.sample(n=3000, replace=False, random_state=1)
sample_neg_poverty_lower = label_neg_poverty_lower.sample(n=12000, replace=False, random_state=1)

In [267]:
new_traindf = pd.concat([sample_pos_poverty_highest,sample_neg_poverty_highest,sample_pos_poverty_lower, sample_neg_poverty_lower], axis=0)
y_train = new_traindf['quickstart_label'].values
rf.fit(new_traindf.drop(['entity_id','as_of_date','quickstart_label'], axis = 1), y_train)

RandomForestClassifier(max_depth=30, max_features='sqrt', min_samples_leaf=44,
                       min_samples_split=3, n_estimators=87, n_jobs=-1,
                       random_state=213500298)

In [268]:
y_pred = rf.predict_proba(testdf.drop(['entity_id','as_of_date','quickstart_label'], axis = 1))[:,1]
new_preds = testdf[['entity_id','as_of_date','quickstart_label']].copy()
new_preds['predict_proba'] = y_pred
new_preds = new_preds.sort_values('predict_proba', ascending = False).reset_index(drop=True).copy()
new_preds['score'] = new_preds.apply(lambda x: 1.0 if int(x.name)  < 1000 else 0.0, axis=1)

In [269]:
df = pd.merge(new_preds, test_attrdf, how='left', on=['entity_id','as_of_date'], left_index=True, right_index=False, sort=True, copy=True)
df = df.rename(columns = {'quickstart_label':'label_value'})
g = Group()
xtab, _ = g.get_crosstabs(df[['score','label_value','poverty_level','metro_type', 'teacher_sex']].copy())

model_id, score_thresholds 0 {'rank_abs': [1000]}


In [270]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics]

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,poverty_level,highest,0.096401,0.970850,0.296844,0.400000,0.029150,0.903599,0.703156,0.600000,0.500,0.050135,0.312043
1,poverty_level,lower,0.092022,0.951094,0.360217,0.474000,0.048906,0.907978,0.639783,0.526000,0.500,0.064901,0.370976
2,metro_type,suburban_rural,0.123694,0.937029,0.352378,0.466667,0.062971,0.876306,0.647622,0.533333,0.555,0.085306,0.367814
3,metro_type,unknown,0.120261,0.948877,0.354584,0.417722,0.051123,0.879739,0.645416,0.582278,0.158,0.076848,0.372082
4,metro_type,urban,0.062233,0.982231,0.298709,0.390244,0.017769,0.937767,0.701291,0.609756,0.287,0.031487,0.308502
5,teacher_sex,female,0.104721,0.957873,0.325999,0.437370,0.042127,0.895279,0.674001,0.562630,0.958,0.063473,0.341019
6,teacher_sex,male,0.029162,0.989779,0.314319,0.428571,0.010221,0.970838,0.685681,0.571429,0.042,0.016254,0.318498


In [274]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict={'poverty_level':'lower', 'metro_type':'suburban_rural', 'teacher_sex':'male'})
metrics = ['tpr']
ap.disparities(bdf, metrics, 'poverty_level', fairness_threshold = 1.3)

get_disparity_predefined_group()


alt.LayerChart(...)

In [12]:
df = df.sort_values('quickstart_label', ascending=False)

In [13]:
teacher_female_df = df[df['teacher_sex']=='female'].copy()

In [ ]:
df_pov_high = df[df['poverty']]

In [14]:
teacher_male_df = df[df['teacher_sex']=='male'].copy()

In [15]:
teacher_female_df.head()

,entity_id,as_of_date,quickstart_label,metro_type,grade_level,poverty_level,teacher_sex
98745,171,1343775600000,1.0,urban,PreK-2,high,female
90235,243783,1340146800000,1.0,suburban,PreK-2,low_moderate,female
90222,243770,1340146800000,1.0,rural,9-12,high,female
90223,243771,1340146800000,1.0,urban,PreK-2,high,female
90232,243780,1340146800000,1.0,rural,PreK-2,high,female


In [31]:
teacher_female_df.shape


(14994, 8)

In [30]:
teacher_male_df.shape

(1796, 8)

In [16]:
teacher_male_df.head()

,entity_id,as_of_date,quickstart_label,metro_type,grade_level,poverty_level,teacher_sex
90229,243777,1340146800000,1.0,rural,3-5,high,male
90233,243781,1340146800000,1.0,unknown,3-5,high,male
90260,243740,1340146800000,1.0,urban,PreK-2,high,male
90013,243856,1340060400000,1.0,urban,6-8,high,male
90011,243854,1340060400000,1.0,unknown,9-12,high,male


In [17]:
df.shape

(16790, 7)

In [18]:
teacher_male_df['cumsum_prev'] = teacher_male_df['quickstart_label'].cumsum() / len(teacher_male_df)

In [19]:
teacher_female_df['cumsum_prev'] = teacher_female_df['quickstart_label'].cumsum() / len(teacher_female_df)

In [20]:
teacher_female_df.head()

,entity_id,as_of_date,quickstart_label,metro_type,grade_level,poverty_level,teacher_sex,cumsum_prev
98745,171,1343775600000,1.0,urban,PreK-2,high,female,0.000067
90235,243783,1340146800000,1.0,suburban,PreK-2,low_moderate,female,0.000133
90222,243770,1340146800000,1.0,rural,9-12,high,female,0.000200
90223,243771,1340146800000,1.0,urban,PreK-2,high,female,0.000267
90232,243780,1340146800000,1.0,rural,PreK-2,high,female,0.000333


In [21]:
prev_df = pd.concat([teacher_male_df, teacher_female_df], axis=0).sort_values('cumsum_prev', ascending=True)

In [22]:
prev_df.head()

,entity_id,as_of_date,quickstart_label,metro_type,grade_level,poverty_level,teacher_sex,cumsum_prev
98745,171,1343775600000,1.0,urban,PreK-2,high,female,0.000067
90235,243783,1340146800000,1.0,suburban,PreK-2,low_moderate,female,0.000133
90222,243770,1340146800000,1.0,rural,9-12,high,female,0.000200
90223,243771,1340146800000,1.0,urban,PreK-2,high,female,0.000267
90232,243780,1340146800000,1.0,rural,PreK-2,high,female,0.000333


In [25]:
new_prev = prev_df.head(10000).copy()

In [26]:
new_prev.tail(10)

,entity_id,as_of_date,quickstart_label,metro_type,grade_level,poverty_level,teacher_sex,cumsum_prev
90729,243015,1340492400000,0.0,unknown,3-5,high,female,0.384087
90730,243016,1340492400000,0.0,urban,PreK-2,high,female,0.384087
90731,243017,1340492400000,0.0,urban,PreK-2,high,female,0.384087
90732,243018,1340492400000,0.0,rural,9-12,high,female,0.384087
90733,243019,1340492400000,0.0,urban,3-5,high,female,0.384087
90748,243034,1340492400000,0.0,rural,6-8,high,female,0.384087
95567,238441,1342738800000,0.0,urban,9-12,high,female,0.384087
90694,243010,1340492400000,0.0,urban,PreK-2,high,female,0.384087
90632,243025,1340492400000,0.0,urban,9-12,high,female,0.384087
90740,243026,1340492400000,0.0,unknown,3-5,high,female,0.384087


In [27]:
new_prev['teacher_sex'].value_counts()

female    8204
male      1796
Name: teacher_sex, dtype: int64

In [ ]:
new_prev.groupby()

In [131]:
new_preds = testdf[['entity_id','as_of_date','quickstart_label']].copy()